# Subconsultas e expressões de tabela comuns

Espero que agora você já esteja familiarizado com junções, particularmente com `INNER JOIN` e `LEFT JOIN`. Embora junções sejam uma habilidade essencial, o SQL se torna incrivelmente flexível e poderoso quando você aprende sobre subconsultas, tabelas derivadas e expressões comuns de tabelas. Ao usar essas operações, você poderá descobrir que o SQL pode expressar lógica e tarefas declarativamente que você talvez não acreditasse serem possíveis.

## Configuração

Primeiro, configure. Baixe o arquivo de banco de dados SQLite `company_operations.db` e conecte-se a ele. Também inclua `pandas` para exibir os resultados da nossa consulta SQL como um `DataFrame`.

In [ ]:
import sqlite3
import pandas as pd
import urllib.request

# baixe o banco de dados SQLite e conecte-se a ele
urllib.request.urlretrieve("https://github.com/thomasnield/anaconda_intro_to_sql/blob/main/company_operations.db?raw=true", "company_operations.db")
conn = sqlite3.connect('company_operations.db')

## Subconsultas escalares

Vamos encontrar o máximo `ORDER_DATE` que existe na tabela `CUSTOMER_ORDER`.

In [ ]:
sql = """
SELECT MAX(ORDER_DATE) FROM CUSTOMER_ORDER
"""

pd.read_sql(sql, conn)


Agora, digamos que queremos obter todos os registros `CUSTOMER_ORDER` para a última `ORDER_DATE`. Em vez de codificar esse valor como um literal, podemos incorporar essa primeira consulta como uma **subconsulta**, que é uma consulta dentro de outra consulta. Nesse caso, trata-se de uma **subconsulta escalar**, pois retorna um único valor.

In [ ]:
sql = """
SELECT * FROM CUSTOMER_ORDER
WHERE ORDER_DATE = (SELECT MAX(ORDER_DATE) FROM CUSTOMER_ORDER)
"""

pd.read_sql(sql, conn)


## Subconsultas de matriz

Digamos que você queira obter todos os registros `CUSTOMER_ORDER` para clientes que estão no `ESTADO` de "TX". Poderíamos fazer isso usando um `INNER JOIN`, mas vamos tentar fazer isso de uma forma um pouco mais simples (e possivelmente mais eficiente).

Primeiro, vamos obter uma única coluna de `CUSTOMER_ID` para clientes no `ESTADO` de "TX".

In [ ]:
sql = """
SELECT CUSTOMER_ID FROM CUSTOMER WHERE STATE = 'TX'
"""

pd.read_sql(sql, conn)


Podemos então incorporar isso como uma subconsulta em uma operação `IN`. Isso é conhecido como uma **subconsulta de array** porque retorna uma lista de valores.

In [ ]:
sql = """
SELECT * FROM CUSTOMER_ORDER

WHERE CUSTOMER_ID IN (SELECT CUSTOMER_ID FROM CUSTOMER WHERE STATE = 'TX')
"""

pd.read_sql(sql, conn)


Outra tarefa comum para subconsultas de array é obter registros pai sem filhos, como registros `CUSTOMER` sem registros `CUSTOMER_ORDER`. Podemos qualificar um conjunto `DISTINCT` de valores `CUSTOMER_ID` da tabela `CUSTOMER_ORDER` (removendo quaisquer duplicatas) e verificar se há registros `CUSTOMER` cujo `CUSTOMER_ID` não esteja presente.

Nesse caso, devemos obter apenas um `CUSTOMER` que não tenha nenhum registro `CUSTOMER_ORDER`.

In [ ]:
sql = """
SELECT * FROM CUSTOMER

WHERE CUSTOMER_ID NOT IN (SELECT DISTINCT CUSTOMER_ID FROM CUSTOMER_ORDER)
"""

pd.read_sql(sql, conn)


## Subconsultas correlacionadas

Podemos usar **subconsultas correlacionadas** para que uma subconsulta faça referência aos campos da consulta externa. Por exemplo, podemos mostrar os registros `CUSTOMER_ORDER`, mas também calcular a quantidade média pedida para todos os registros que compartilham `CUSTOMER_ID` e `PRODUCT_ID`. Observe que `CUSTOMER_ORDER` está sendo usado em dois contextos: a consulta interna, com o alias `co2`, e a consulta externa, com o alias `co1`.

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID, 
CUSTOMER_ID,
ORDER_DATE,
PRODUCT_ID,
QUANTITY,

(
  SELECT AVG(QUANTITY)
  FROM CUSTOMER_ORDER co2 
  WHERE co1.CUSTOMER_ID = co2.CUSTOMER_ID 
  AND co1.PRODUCT_ID = co2.PRODUCT_ID
) AS AVG_QTY

FROM CUSTOMER_ORDER co1

"""

pd.read_sql(sql, conn)


Esta não é, de forma alguma, a maneira mais eficiente de realizar esta tarefa, e aprenderemos algumas maneiras melhores de realizar esta tarefa específica de obter um valor agregado que compartilhe os atributos de cada registro. No entanto, subconsultas correlacionadas podem ser uma ferramenta poderosa para calcular com flexibilidade outras consultas dependentes de cada registro. Observe que isso é computacionalmente custoso, pois cada registro iniciará esta subconsulta.

## Tabelas Derivadas

Quando uma subconsulta contém várias colunas, a chamamos de **tabela derivada**. Isso costuma ser usado para declarar uma consulta `SELECT` e uni-la como se fosse uma tabela. Observe abaixo como podemos exibir a quantidade média pedida ao lado de cada registro `CUSTOMER_ORDER`, para todos os registros que compartilham o `CUSTOMER_ID` e o `PRODUCT_ID` de cada registro.

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID, 
CUSTOMER_ORDER.CUSTOMER_ID,
ORDER_DATE,
CUSTOMER_ORDER.PRODUCT_ID,
QUANTITY,
AVG_QTY

FROM CUSTOMER_ORDER LEFT JOIN 

(
  SELECT CUSTOMER_ID, 
  PRODUCT_ID,
  AVG(QUANTITY) AS AVG_QTY

  FROM CUSTOMER_ORDER
  GROUP BY CUSTOMER_ID, PRODUCT_ID
) avg_quantity

ON CUSTOMER_ORDER.CUSTOMER_ID = avg_quantity.CUSTOMER_ID
AND CUSTOMER_ORDER.PRODUCT_ID = avg_quantity.PRODUCT_ID

"""

pd.read_sql(sql, conn)


Isso é muito mais eficiente, pois calculamos a `QUANTIDADE` média para cada `PRODUCT_ID` e `CUSTOMER_ID` de uma só vez e juntamos tudo.

## Expressões de tabela comuns (CTE's)

Uma maneira muito melhor de declarar tabelas derivadas é declarar uma **expressão de tabela comum**, que permite declarar uma subconsulta nomeada com antecedência, antes de usá-la em uma consulta `SELECT`. Vamos pegar o exemplo anterior e observá-lo em uma expressão de tabela comum.

In [ ]:
sql = """
WITH avg_quantity AS (
  SELECT CUSTOMER_ID, 
  PRODUCT_ID,
  AVG(QUANTITY) AS AVG_QTY

  FROM CUSTOMER_ORDER
  GROUP BY CUSTOMER_ID, PRODUCT_ID
)

SELECT CUSTOMER_ORDER_ID, 
CUSTOMER_ORDER.CUSTOMER_ID,
ORDER_DATE,
CUSTOMER_ORDER.PRODUCT_ID,
QUANTITY,
AVG_QTY

FROM CUSTOMER_ORDER LEFT JOIN avg_quantity

ON CUSTOMER_ORDER.CUSTOMER_ID = avg_quantity.CUSTOMER_ID
AND CUSTOMER_ORDER.PRODUCT_ID = avg_quantity.PRODUCT_ID

"""

pd.read_sql(sql, conn)


Os benefícios são principalmente a reutilização e a organização do código. Podemos usar uma expressão de tabela comum várias vezes sem precisar declarar novamente sua consulta `SELECT` de forma redundante. Também podemos evitar a confusão de consultas `SELECT` dentro de consultas `SELECT` e dividir a consulta em etapas mais fáceis de entender.

Outro benefício é que você pode ter, proceduralmente, várias expressões de tabela comuns, onde cada uma pode apontar para a anterior. Abaixo, declaramos `tx_customer_ids` para obter os IDs de clientes no estado `TX` e, em seguida, usamos isso para obter pedidos apenas de clientes no estado `TX`. Por fim, reduzimos esses pedidos para apenas `PRODUCT_ID` de 7.

In [ ]:
sql = """
WITH tx_customer_ids AS (
  SELECT CUSTOMER_ID 
  FROM CUSTOMER
  WHERE STATE = 'TX'
), 

tx_customer_orders AS (
  SELECT * FROM CUSTOMER_ORDER 
  WHERE CUSTOMER_ID IN tx_customer_ids
)

SELECT * FROM tx_customer_orders
WHERE PRODUCT_ID = 7 

"""

pd.read_sql(sql, conn)


Embora este exemplo possa dividir essas etapas desnecessariamente, ele serve para mostrar que você pode dividir consultas mais complexas em etapas simples.

## EXERCÍCIO

Para cada `CUSTOMER_ORDER` no mês de março, recupere todos os campos. Insira também as quantidades mínima e máxima de produtos encomendados em todos os registros que compartilham `PRODUCT_ID` e `CUSTOMER_ID` de cada registro.

In [ ]:
sql = """
? min_max_quantity AS (
  ?
)

SELECT CUSTOMER_ORDER_ID, 
CUSTOMER_ORDER.CUSTOMER_ID,
ORDER_DATE,
CUSTOMER_ORDER.PRODUCT_ID,
QUANTITY,
MIN_QTY,
MAX_QTY

FROM CUSTOMER_ORDER LEFT JOIN ?

ON CUSTOMER_ORDER.CUSTOMER_ID = min_max_quantity.CUSTOMER_ID
AND CUSTOMER_ORDER.PRODUCT_ID = min_max_quantity.PRODUCT_ID

"""

pd.read_sql(sql, conn)


### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
sql = """
WITH min_max_quantity AS (
  SELECT CUSTOMER_ID, 
  PRODUCT_ID,
  MIN(QUANTITY) AS MIN_QTY,
  MAX(QUANTITY) AS MAX_QTY

  FROM CUSTOMER_ORDER
  GROUP BY CUSTOMER_ID, PRODUCT_ID
)

SELECT CUSTOMER_ORDER_ID, 
CUSTOMER_ORDER.CUSTOMER_ID,
ORDER_DATE,
CUSTOMER_ORDER.PRODUCT_ID,
QUANTITY,
MIN_QTY,
MAX_QTY

FROM CUSTOMER_ORDER LEFT JOIN min_max_quantity

ON CUSTOMER_ORDER.CUSTOMER_ID = min_max_quantity.CUSTOMER_ID
AND CUSTOMER_ORDER.PRODUCT_ID = min_max_quantity.PRODUCT_ID

"""

pd.read_sql(sql, conn)
